## Charting for Monitor

This notebook walks through building charts used in the Monitor application.

To run this notebook, start Jupyter as follows from the Django project root: <br>
(https://medium.com/ayuth/how-to-use-django-in-jupyter-notebook-561ea2401852)

`python manage.py shell_plus --notebook`

<br>
NOTE:  You will need to changed the kernel from menu: `Kernel` > `Change kernel` > `Django Shell-Plus`

In [130]:
import os
import logging
import pandas as pd
from pandas import Grouper
from pygam import LinearGAM, s


In [2]:


try:
    from django.db import models
    from traffic_monitor.models.model_logentry import LogEntry
    

except Exception as e:
    print("NOT LOADED:  start notebook with:\n")
    print("\tpython manage.py shell_plus --notebook")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from bokeh.io import output_notebook, show
from bokeh.models import LinearColorMapper, ColorBar, FixedTicker, NumeralTickFormatter

from bokeh.models import HoverTool, WheelZoomTool, ResetTool
from bokeh.models import Legend, LegendItem
from bokeh.plotting import figure
from bokeh.palettes import brewer, RdYlGn11
from bokeh.embed import components
from bokeh.embed import json_item
from bokeh.models import ColumnDataSource, HTMLTemplateFormatter, TableColumn, DataTable, HoverTool, \
    DatetimeTickFormatter, Line, FactorRange
from bokeh.plotting import figure
from django.http import JsonResponse
output_notebook()

Loading BokehJS ...

In [3]:
logger = logging.getLogger("model_logger")
logger.setLevel(logging.INFO)

In [4]:
monitor_id=6
class_ids: list = ['car', 'truck', 'bus', 'person', 'motorcycle']
rs = LogEntry.objects.filter(monitor_id=monitor_id).values('time_stamp', 'class_id', 'count')
df = pd.DataFrame(rs)


In [233]:
def get_chart(monitor_id: int, interval: int = None):
    tools = [HoverTool(
        tooltips=[
#             ("Time", "@time_stamp{%m/%d/%y %T}"), # https://docs.bokeh.org/en/latest/docs/reference/models/formatters.html#bokeh.models.formatters.DatetimeTickFormatter
#             ("Class", '@class_id')
            ("Time", "$x{%m/%d/%y %T}"), # https://docs.bokeh.org/en/latest/docs/reference/models/formatters.html#bokeh.models.formatters.DatetimeTickFormatter
            ("Rate", '$y'),
        ],
        formatters={'$x': 'datetime'}
#         , mode='vline'
    )]
    
    # data
    rs = LogEntry.objects.filter(monitor_id=monitor_id).values('time_stamp', 'class_id', 'count')
    df = pd.DataFrame(rs)
    df.rename(columns={'count':'counts'}, inplace=True)
    
    # set timezone
    df = df.set_index('time_stamp').tz_convert('US/Mountain').tz_localize(None)
    
    # get top 5 items
    top5_classes = df.groupby('class_id').count().sort_values(by='counts', ascending=False).index[0:5].values
    df = df[df['class_id'].isin(top5_classes)]
    df.sort_index(inplace=True)
    
    if interval:
        df = df.groupby(by=['class_id', Grouper(key='time_stamp', freq=f'{interval}min')]).mean()

    # Define the blank canvas of the Bokeh plot that data will be layered on top of
    fig = figure(
                 sizing_mode="stretch_both",
                 tools=tools,
                 toolbar_location=None,
                 x_axis_type="datetime",
                 border_fill_color=None,
                 min_border_left=0)

    # Remove classic x and y ticks and chart junk to make things clean
    fig.xgrid.grid_line_color = None
    fig.xaxis.axis_line_color = None
    fig.yaxis.axis_line_color = None
    fig.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    fig.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    fig.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    fig.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks

    # Hide hours and minutes in the x-axis
    fig.xaxis.formatter = DatetimeTickFormatter(
                                                days="%m/%d/%Y",
                                                months="%m/%Y",
                                                hours="%H:%M",
                                                minutes="",
                                                seconds="%m/%d/%Y %H:%M:%S")

    # create plot lines for each class
    u_class_ids = sorted(df.class_id.unique())
    df = df[df.class_id.isin(u_class_ids)]
    colors = brewer['Spectral'][5]
    df['fill_color'] = [colors[u_class_ids.index(c)] for c in df.class_id]
    
    
# Multiple Lines
    for i, class_id in enumerate(top5_classes):
        _df = df[df.class_id == class_id].sort_index()
        _x = (_df.index - _df.index.min()).astype(int)

        _df['smoothed_counts'] = LinearGAM(s(0, lam=1)).fit(_x, _df.counts).predict(_x)

        fig.line(x='time_stamp', y='smoothed_counts', source=_df,
                 legend_label=class_id, color=colors[i], line_width=1.5)
        fig.scatter(x='time_stamp', y='counts', source=_df,
                 color=colors[i], size=2, alpha=.5)

#     fig.add_layout(legend)
    fig.legend.location = "top_left"
    fig.legend.spacing = 0
    fig.legend.padding = 5

# Line Plot
#     xs = [df[df.class_id==c]['time_stamp'] for c in u_class_ids]
#     ys = [df[df.class_id==c]['counts'] for c in u_class_ids]
#     r = fig.multi_line(xs, ys, line_color=colors[0:len(u_class_ids)])
#     legend = Legend(items=[
#         LegendItem(label=c, renderers=[r], index=u_class_ids.index(c)) for c in u_class_ids
#     ])
#     fig.add_layout(legend)
    
# Scatter Plot    
#     r = fig.circle('time_stamp', 'counts', size=3, fill_color='fill_color', legend_field='class_id', line_color=None, source=df)

# Multiline
#     ds = ColumnDataSource({ 'xs': [df[df.class_id==c]['time_stamp'] for c in u_class_ids],
#                             'ys': [df[df.class_id==c]['counts'] for c in u_class_ids],
#                             'class_id': [df[df.class_id==c]['class_id'] for c in u_class_ids],
#                             'label': u_class_ids, 
#                             'color': colors})
#     r = fig.multi_line('xs', 'ys', color='color', source=ds)
#     legend = Legend(items=[
#         LegendItem(label=c, renderers=[r], index=u_class_ids.index(c)) for c in u_class_ids
#     ])
#     fig.add_layout(legend)
#     fig.legend.location = "top_left"
#     fig.legend.spacing = 0
    
    # jupyter notebook
    return _df, components(fig)

#     # Django
#     item = json_item(fig)
#     return JsonResponse(item)



In [234]:
_df, (script, div) = get_chart(monitor_id=6, interval=None)
# show(fig)

In [235]:
from IPython.core.display import display, HTML
display(HTML(script), HTML(div))

In [220]:
_df.index.tz_localize(None)

DatetimeIndex(['2020-06-04 06:21:02.017627', '2020-06-04 07:12:39.890791',
               '2020-06-04 07:55:50.296588', '2020-06-04 07:59:50.682776',
               '2020-06-04 08:11:51.470553', '2020-06-04 08:20:51.861542',
               '2020-06-04 09:02:54.399179', '2020-06-04 09:38:56.327637',
               '2020-06-04 11:44:02.830916', '2020-06-04 11:45:02.855725',
               '2020-06-04 11:59:03.439309', '2020-06-04 12:00:03.464495',
               '2020-06-04 12:17:04.512115', '2020-06-04 12:28:57.300361',
               '2020-06-04 12:32:57.440884', '2020-06-04 12:33:57.469125',
               '2020-06-04 12:36:57.603537', '2020-06-04 12:37:57.629603',
               '2020-06-04 12:38:57.665968', '2020-06-04 12:39:57.700008',
               '2020-06-04 12:43:57.860258', '2020-06-04 12:47:58.128841',
               '2020-06-04 12:48:58.162705', '2020-06-04 12:51:58.323828',
               '2020-06-04 12:57:58.641039', '2020-06-04 12:58:58.661756',
               '2020-06-0

In [194]:
_df.head()


,class_id,counts,fill_color,smoothed_counts
time_stamp,,,,
2020-06-04 06:21:02.017627-06:00,motorcycle,0.083,#ffffbf,0.079719
2020-06-04 07:12:39.890791-06:00,motorcycle,0.083,#ffffbf,0.079650
2020-06-04 07:55:50.296588-06:00,motorcycle,0.083,#ffffbf,0.087748
2020-06-04 07:59:50.682776-06:00,motorcycle,0.077,#ffffbf,0.089059
2020-06-04 08:11:51.470553-06:00,motorcycle,0.077,#ffffbf,0.093504


In [199]:
ds = ColumnDataSource(_df)

In [207]:
ds.data['times_tamp'] = ds.data['times_tamp'].tz_convert('US/Mountain')

In [203]:
ds

ColumnDataSource(id='10327', ...)

In [208]:
ds.data

{'time_stamp': array(['2020-06-04T12:21:02.017627000', '2020-06-04T13:12:39.890791000',
        '2020-06-04T13:55:50.296588000', '2020-06-04T13:59:50.682776000',
        '2020-06-04T14:11:51.470553000', '2020-06-04T14:20:51.861542000',
        '2020-06-04T15:02:54.399179000', '2020-06-04T15:38:56.327637000',
        '2020-06-04T17:44:02.830916000', '2020-06-04T17:45:02.855725000',
        '2020-06-04T17:59:03.439309000', '2020-06-04T18:00:03.464495000',
        '2020-06-04T18:17:04.512115000', '2020-06-04T18:28:57.300361000',
        '2020-06-04T18:32:57.440884000', '2020-06-04T18:33:57.469125000',
        '2020-06-04T18:36:57.603537000', '2020-06-04T18:37:57.629603000',
        '2020-06-04T18:38:57.665968000', '2020-06-04T18:39:57.700008000',
        '2020-06-04T18:43:57.860258000', '2020-06-04T18:47:58.128841000',
        '2020-06-04T18:48:58.162705000', '2020-06-04T18:51:58.323828000',
        '2020-06-04T18:57:58.641039000', '2020-06-04T18:58:58.661756000',
        '2020-06-04T18:5